<a href="https://colab.research.google.com/github/chandreshraghuwanshi123456-jpg/ML_salary_predictor/blob/main/Copy_of_Untitled18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense
import shutil

# --- 1. CONFIGURATION ---
ZIP_NAME = "jump.zip"  # The file already uploaded
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20

# --- 2. UNZIP DATA ---
print(f"📂 Looking for {ZIP_NAME}...")
extract_path = "Training_Data"

if os.path.exists(extract_path):
    shutil.rmtree(extract_path)

if not os.path.exists(ZIP_NAME):
    print(f"❌ ERROR: Could not find '{ZIP_NAME}' in the Files sidebar.")
    print("👉 Please drag and drop 'jump.zip' into the sidebar on the left.")
else:
    with zipfile.ZipFile(ZIP_NAME, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Unzip Complete!")

    # --- 3. AUTO-DETECT CLASSES ---
    # We look inside the extracted folder to find class names automatically
    CLASSES_LIST = []
    for entry in os.listdir(extract_path):
        full_path = os.path.join(extract_path, entry)
        if os.path.isdir(full_path):
            CLASSES_LIST.append(entry)

    CLASSES_LIST.sort() # Ensure consistent order
    print(f"✅ Found classes: {CLASSES_LIST}")

    if len(CLASSES_LIST) < 2:
        print("⚠️ WARNING: Found fewer than 2 classes. Training might fail or be useless.")

    # --- 4. PREPARE DATA ---
    def frames_extraction(video_path):
        frames_list = []
        video_reader = cv2.VideoCapture(video_path)
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

        for frame_counter in range(SEQUENCE_LENGTH):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            success, frame = video_reader.read()
            if not success: break
            resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
            normalized_frame = resized_frame / 255
            frames_list.append(normalized_frame)
        video_reader.release()
        return frames_list

    def create_dataset():
        features, labels = [], []
        print("⚙️ Processing videos...")

        for class_index, class_name in enumerate(CLASSES_LIST):
            class_folder = os.path.join(extract_path, class_name)
            if not os.path.isdir(class_folder): continue

            print(f"   -> Processing class: {class_name}")
            files = os.listdir(class_folder)

            for file_name in files:
                if file_name.lower().endswith(('.avi', '.mp4')):
                    video_path = os.path.join(class_folder, file_name)
                    frames = frames_extraction(video_path)
                    if len(frames) == SEQUENCE_LENGTH:
                        features.append(frames)
                        labels.append(class_index)
        return np.asarray(features), np.array(labels)

    features, labels = create_dataset()

    if len(features) == 0:
        print("❌ ERROR: No videos found. Check folders inside zip.")
    else:
        one_hot_encoded_labels = to_categorical(labels)
        print(f"✅ Data Ready! {len(features)} videos found.")

        # --- 5. BUILD & TRAIN MODEL ---
        print("🧠 Building Neural Network...")
        model = Sequential()
        model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                            return_sequences=True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
        model.add(TimeDistributed(Dropout(0.2)))
        model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', data_format="channels_last", return_sequences=True))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
        model.add(Flatten())
        model.add(Dense(len(CLASSES_LIST), activation="softmax"))

        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])

        print("🏋️ Starting Training (40 Epochs)...")
        model.fit(features, one_hot_encoded_labels, epochs=40, batch_size=4, shuffle=True, validation_split=0.2)

        # --- 6. SAVE ---
        model_name = "Final_Model.keras"
        model.save(model_name)
        print("-" * 30)
        print(f"🎉 SUCCESS! The model is saved as '{model_name}'.")
        print("👉 Right-click 'Final_Model.keras' in the Files sidebar -> Download.")

📂 Looking for jump.zip...
✅ Unzip Complete!
✅ Found classes: ['jump', 'run', 'walk']
⚙️ Processing videos...
   -> Processing class: jump
   -> Processing class: run
   -> Processing class: walk
✅ Data Ready! 29 videos found.
🧠 Building Neural Network...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🏋️ Starting Training (40 Epochs)...
Epoch 1/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.4849 - loss: 2.0356 - val_accuracy: 0.0000e+00 - val_loss: 0.8541
Epoch 2/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4695 - loss: 1.3012 - val_accuracy: 0.0000e+00 - val_loss: 2.7637
Epoch 3/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.4474 - loss: 1.1827 - val_accuracy: 0.0000e+00 - val_loss: 1.1708
Epoch 4/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5207 - loss: 1.0820 - val_accuracy: 0.0000e+00 - val_loss: 1.5525
Epoch 5/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.4752 - loss: 1.0082 - val_accuracy: 0.0000e+00 - val_loss: 1.6321
Epoch 6/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6068 - loss: 1.0061 - val_accuracy: 0.0000e+00 - val_loss: 1.4378
Epoch 7/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3860 - loss: 1.0424 - val_accuracy: 0.0000e+00 - val_loss: 1.3606
Epoch 8/40
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, MaxPooling3D, TimeDistributed, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import shutil

# ==========================================
# PART 1: CONFIGURATION & TRAINING
# ==========================================

# ⚠️ Change this to match your uploaded file!
ZIP_NAME = "smart_subset.zip"
CLASSES_LIST = ["Clapping", "Sitting", "Standing Still", "Walking"]
# (If using jump.zip, change to ["jump", "run", "walk"])

IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20

# 1. Unzip
print(f"📂 Unzipping {ZIP_NAME}...")
extract_path = "Training_Data"
if os.path.exists(extract_path): shutil.rmtree(extract_path)

if not os.path.exists(ZIP_NAME):
    print(f"❌ ERROR: Please upload '{ZIP_NAME}' to the Files sidebar first!")
else:
    with zipfile.ZipFile(ZIP_NAME, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("✅ Unzip Complete!")

    # 2. Process Data
    def frames_extraction(video_path):
        frames_list = []
        video_reader = cv2.VideoCapture(video_path)
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)
        for frame_counter in range(SEQUENCE_LENGTH):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            success, frame = video_reader.read()
            if not success: break
            resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
            normalized_frame = resized_frame / 255
            frames_list.append(normalized_frame)
        video_reader.release()
        return frames_list

    def create_dataset():
        features, labels = [], []
        print("🔍 Scanning folders...")
        for class_index, class_name in enumerate(CLASSES_LIST):
            found = False
            for root, dirs, files in os.walk(extract_path):
                if os.path.basename(root).lower() == class_name.lower():
                    print(f"   -> Processing: {class_name}")
                    for file_name in files:
                        if file_name.lower().endswith(('.avi', '.mp4')):
                            video_path = os.path.join(root, file_name)
                            frames = frames_extraction(video_path)
                            if len(frames) == SEQUENCE_LENGTH:
                                features.append(frames)
                                labels.append(class_index)
                                found = True
            if not found: print(f"⚠️ Warning: No videos found for {class_name}")
        return np.asarray(features), np.array(labels)

    features, labels = create_dataset()

    if len(features) == 0:
        print("❌ CRITICAL ERROR: No data found. Check ZIP_NAME and CLASSES_LIST.")
    else:
        one_hot_encoded_labels = to_categorical(labels)
        print(f"✅ Data Ready! {len(features)} videos.")

        # 3. Build & Train
        print("🧠 Training Model...")
        model = Sequential()
        model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                            return_sequences=True, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
        model.add(TimeDistributed(Dropout(0.2)))
        model.add(ConvLSTM2D(filters=8, kernel_size=(3, 3), activation='tanh', data_format="channels_last", return_sequences=True))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
        model.add(Flatten())
        model.add(Dense(len(CLASSES_LIST), activation="softmax"))

        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
        model.fit(features, one_hot_encoded_labels, epochs=30, batch_size=4, shuffle=True)
        model.save("Colab_Brain.keras")
        print("🎉 TRAINING SUCCESS! Model saved.")

# ==========================================
# PART 2: LIVE WEBCAM DEMO IN COLAB
# ==========================================

def start_webcam():
  js = Javascript('''
    async function createWebcam() {
      const div = document.createElement('div');
      document.body.appendChild(div);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Create a canvas to grab the frame
      const canvas = document.createElement('canvas');
      canvas.width = 640;
      canvas.height = 480;

      div.appendChild(video);

      // Label Element
      const label = document.createElement('div');
      label.textContent = "Loading AI...";
      label.style.fontSize = "24px";
      label.style.fontWeight = "bold";
      label.style.color = "red";
      div.appendChild(label);

      return {video, canvas, label};
    }

    var webcam = null;

    async function captureFrame() {
      if (!webcam) {
        webcam = await createWebcam();
      }

      // Draw video to canvas
      const ctx = webcam.canvas.getContext('2d');
      ctx.drawImage(webcam.video, 0, 0, 640, 480);

      // Convert to base64
      const result = webcam.canvas.toDataURL('image/jpeg', 0.5);
      return result;
    }

    function updateLabel(text) {
        if (webcam && webcam.label) {
            webcam.label.textContent = text;
            webcam.label.style.color = "green";
        }
    }
    ''')
  display(js)

def js_to_image(js_reply):
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)
  return img

# START THE DEMO
print("\n🎥 Starting Webcam Demo...")
start_webcam()

frames_queue = []
import time

while True:
    try:
        # Get frame from JS
        js_reply = eval_js('captureFrame()')
        if not js_reply: break

        frame = js_to_image(js_reply)

        # Preprocess
        resized = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized = resized / 255.0
        frames_queue.append(normalized)

        if len(frames_queue) == SEQUENCE_LENGTH:
            input_data = np.expand_dims(np.array(frames_queue), axis=0)
            prob = model.predict(input_data, verbose=0)[0]
            pred_idx = np.argmax(prob)
            action = CLASSES_LIST[pred_idx]
            conf = prob[pred_idx]

            # Send result back to JS to display on screen
            status = f"ACTION: {action.upper()} ({conf*100:.1f}%)"
            eval_js(f'updateLabel("{status}")')

            frames_queue = frames_queue[5:]

    except Exception as e:
        print(f"Error: {e}")
        break

📂 Unzipping smart_subset.zip...
❌ ERROR: Please upload 'smart_subset.zip' to the Files sidebar first!

🎥 Starting Webcam Demo...


<IPython.core.display.Javascript object>